# Finding Percent of Sims to Take Pathways

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from vivarium import InteractiveContext, Artifact

from datetime import datetime, timedelta
from pathlib import Path
import itertools
import matplotlib.pyplot as plt
import ipywidgets
import pandas as pd, numpy as np
pd.set_option('display.max_rows', 100)

import numpy as np
import researchpy as rp
import statsmodels.api as sm
import statsmodels.formula.api as smf
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns

from db_queries import get_outputs as go
from db_queries import get_ids
from get_draws.api import get_draws

### Create interactive sim data 
#### Note: I have saved an old version of this data, you can choose to skip this step and just load the dataset below

In [2]:
path = Path('/ihme/homes/lutzes/vivarium_nih_us_cvd/src/vivarium_nih_us_cvd/model_specifications/nih_us_cvd.yaml')
path

PosixPath('/ihme/homes/lutzes/vivarium_nih_us_cvd/src/vivarium_nih_us_cvd/model_specifications/nih_us_cvd.yaml')

In [3]:
sim = InteractiveContext(Path(path), setup=False)
sim.configuration.update({
                          'population':
                              {'population_size': 10_000,
                              },
                          'time':
                              {'start':
                                  {'year': 2021
                                  }
                              },
                          'input_data':
                              {'input_draw_number': 829
                              },
                          'intervention':
                              {'scenario': "outreach_100"
                              },
                          'outreach_scale_up':
                              {'value': 
                               {'end': 1 
                               }
                              }
                          }
                        )
sim.setup()

2023-08-22 14:19:00.454 | INFO     | simulation_1-artifact_manager:66 - Running simulation from artifact located at /mnt/team/simulation_science/costeffectiveness/artifacts/vivarium_nih_us_cvd/alabama.hdf.
2023-08-22 14:19:00.456 | INFO     | simulation_1-artifact_manager:67 - Artifact base filter terms are ['draw == 829'].
2023-08-22 14:19:00.458 | INFO     | simulation_1-artifact_manager:68 - Artifact additional filter terms are None.


In [4]:
pop0 = sim.get_population()
#pop0

In [5]:
pop0.columns

Index(['tracked', 'age', 'alive', 'location', 'exit_time', 'sex',
       'entrance_time', 'years_of_life_lost', 'cause_of_death',
       'years_lived_with_disability', 'previous_ischemic_stroke',
       'previous_ischemic_heart_disease_and_heart_failure',
       'high_fasting_plasma_glucose_propensity',
       'high_body_mass_index_in_adults_propensity',
       'high_systolic_blood_pressure_propensity',
       'high_ldl_cholesterol_propensity', 'ischemic_stroke',
       'ischemic_heart_disease_and_heart_failure',
       'ensemble_propensity_risk_factor.high_ldl_cholesterol',
       'ensemble_propensity_risk_factor.high_systolic_blood_pressure',
       'ensemble_propensity_risk_factor.high_body_mass_index_in_adults',
       'ensemble_propensity_risk_factor.high_fasting_plasma_glucose',
       'sbp_medication_adherence_propensity',
       'ldlc_medication_adherence_propensity', 'outreach_propensity',
       'polypill_propensity', 'lifestyle_propensity',
       'susceptible_to_ischemic_st

In [6]:
#sim.list_values()

data1 = pd.concat([pop0,
                   sim.get_value('outreach.exposure')(pop0.index),
                   sim.get_value('lifestyle.exposure')(pop0.index),
                   sim.get_value('polypill.exposure')(pop0.index),
                  ], axis=1)
data1.head()

In [7]:
sim_data = pd.DataFrame()
for step in list(range(0,12)):
    sim.step()
    pop_t = sim.get_population()
    data_t = pd.concat([pop_t, 
                   sim.get_value('high_ldl_cholesterol.exposure')(pop0.index).rename('high_ldl'),
                   sim.get_value('high_systolic_blood_pressure.exposure')(pop0.index).rename('high_sbp'),
                   sim.get_value('high_body_mass_index_in_adults.exposure')(pop0.index).rename('high_bmi'),
                   sim.get_value('high_fasting_plasma_glucose.exposure')(pop0.index).rename('high_fpg'),
                      ], axis=1)
    data_t['step'] = step
    sim_data = pd.concat([sim_data, data_t])
    print(step)

0
1
2
3
4
5
6
7
8
9
10
11


In [8]:
sim_data.to_csv('/ihme/homes/lutzes/sim_graphs_12_steps_outreach_08222023.csv')

In [9]:
sim = InteractiveContext(Path(path), setup=False)
sim.configuration.update({
                          'population':
                              {'population_size': 10_000,
                              },
                          'time':
                              {'start':
                                  {'year': 2021
                                  }
                              },
                          'input_data':
                              {'input_draw_number': 829
                              },
                          'intervention':
                              {'scenario': "polypill_100"
                              },
                          'polypill_scale_up':
                              {'value': 
                               {'end': 1 
                               }
                              }
                          }
                        )
sim.setup()

2023-08-22 14:37:15.637 | INFO     | simulation_2-artifact_manager:66 - Running simulation from artifact located at /mnt/team/simulation_science/costeffectiveness/artifacts/vivarium_nih_us_cvd/alabama.hdf.
2023-08-22 14:37:15.638 | INFO     | simulation_2-artifact_manager:67 - Artifact base filter terms are ['draw == 829'].
2023-08-22 14:37:15.641 | INFO     | simulation_2-artifact_manager:68 - Artifact additional filter terms are None.


In [10]:
sim_data = pd.DataFrame()
for step in list(range(0,12)):
    sim.step()
    pop_t = sim.get_population()
    data_t = pd.concat([pop_t, 
                   sim.get_value('high_ldl_cholesterol.exposure')(pop0.index).rename('high_ldl'),
                   sim.get_value('high_systolic_blood_pressure.exposure')(pop0.index).rename('high_sbp'),
                   sim.get_value('high_body_mass_index_in_adults.exposure')(pop0.index).rename('high_bmi'),
                   sim.get_value('high_fasting_plasma_glucose.exposure')(pop0.index).rename('high_fpg'),
                      ], axis=1)
    data_t['step'] = step
    sim_data = pd.concat([sim_data, data_t])
    print(step)

0
1
2
3
4
5
6
7
8
9
10
11


In [11]:
sim_data.to_csv('/ihme/homes/lutzes/sim_graphs_12_steps_polypill_08222023.csv')

In [12]:
sim = InteractiveContext(Path(path), setup=False)
sim.configuration.update({
                          'population':
                              {'population_size': 10_000,
                              },
                          'time':
                              {'start':
                                  {'year': 2021
                                  }
                              },
                          'input_data':
                              {'input_draw_number': 829
                              },
                          'intervention':
                              {'scenario': "lifestyle_100"
                              },
                          'lifestyle_scale_up':
                              {'value': 
                               {'end': 1 
                               }
                              }
                          }
                        )
sim.setup()

2023-08-22 14:54:03.907 | INFO     | simulation_3-artifact_manager:66 - Running simulation from artifact located at /mnt/team/simulation_science/costeffectiveness/artifacts/vivarium_nih_us_cvd/alabama.hdf.
2023-08-22 14:54:03.908 | INFO     | simulation_3-artifact_manager:67 - Artifact base filter terms are ['draw == 829'].
2023-08-22 14:54:03.909 | INFO     | simulation_3-artifact_manager:68 - Artifact additional filter terms are None.


In [13]:
sim_data = pd.DataFrame()
for step in list(range(0,12)):
    sim.step()
    pop_t = sim.get_population()
    data_t = pd.concat([pop_t, 
                   sim.get_value('high_ldl_cholesterol.exposure')(pop0.index).rename('high_ldl'),
                   sim.get_value('high_systolic_blood_pressure.exposure')(pop0.index).rename('high_sbp'),
                   sim.get_value('high_body_mass_index_in_adults.exposure')(pop0.index).rename('high_bmi'),
                   sim.get_value('high_fasting_plasma_glucose.exposure')(pop0.index).rename('high_fpg'),
                      ], axis=1)
    data_t['step'] = step
    sim_data = pd.concat([sim_data, data_t])
    print(step)

0
1
2
3
4
5
6
7
8
9
10
11


In [14]:
sim_data.to_csv('/ihme/homes/lutzes/sim_graphs_12_steps_lifestyle_08222023.csv')

In [15]:
sim = InteractiveContext(Path(path), setup=False)
sim.configuration.update({
                          'population':
                              {'population_size': 10_000,
                              },
                          'time':
                              {'start':
                                  {'year': 2021
                                  }
                              },
                          'input_data':
                              {'input_draw_number': 829
                              },
                          'intervention':
                              {'scenario': "baseline"
                              },
                          }
                        )
sim.setup()

2023-08-22 15:07:59.915 | INFO     | simulation_4-artifact_manager:66 - Running simulation from artifact located at /mnt/team/simulation_science/costeffectiveness/artifacts/vivarium_nih_us_cvd/alabama.hdf.
2023-08-22 15:07:59.916 | INFO     | simulation_4-artifact_manager:67 - Artifact base filter terms are ['draw == 829'].
2023-08-22 15:07:59.917 | INFO     | simulation_4-artifact_manager:68 - Artifact additional filter terms are None.


In [16]:
sim_data = pd.DataFrame()
for step in list(range(0,12)):
    sim.step()
    pop_t = sim.get_population()
    data_t = pd.concat([pop_t, 
                   sim.get_value('high_ldl_cholesterol.exposure')(pop0.index).rename('high_ldl'),
                   sim.get_value('high_systolic_blood_pressure.exposure')(pop0.index).rename('high_sbp'),
                   sim.get_value('high_body_mass_index_in_adults.exposure')(pop0.index).rename('high_bmi'),
                   sim.get_value('high_fasting_plasma_glucose.exposure')(pop0.index).rename('high_fpg'),
                      ], axis=1)
    data_t['step'] = step
    sim_data = pd.concat([sim_data, data_t])
    print(step)

0
1
2
3
4
5
6
7
8
9
10
11


In [17]:
sim_data.to_csv('/ihme/homes/lutzes/sim_graphs_12_steps_baseline_08222023.csv')

## Analyze Data and Set Up Columns for Graphing 